In [18]:
from __future__ import division
import os, sys
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches

import pandas as pd
from svgpathtools import parse_path, wsvg, svg2paths

### setup

In [19]:
# directory & file hierarchy
proj_dir = os.path.abspath('../../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','python') not in sys.path:
    sys.path.insert(1,os.path.join(proj_dir,'analysis','python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

import svg_rendering_helpers as srh
reload(srh)

<module 'svg_rendering_helpers' from '/Users/megumisano/graphical_conventions/analysis/python/svg_rendering_helpers.pyc'>

In [4]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['graphical_conventions']

# which iteration name should we use?
## "good" iterations that we currently combine: ['run3_size4_waiting', 'run4_generalization']
iterationNames = ['run3_size4_waiting', 'run4_generalization']

### svg rendering

In [6]:
# list of all gameids across iterationNames
gameids = coll.find({'iterationName': {'$in': iterationNames}}).distinct('gameid')

In [7]:
# sample game
this_game = gameids[10]

In [42]:
# sample stroke recs from a trial
this_trial = 1
stroke_recs = coll.find({ '$and': [{'gameid': this_game}, 
                                   {'eventType': 'stroke'}, 
                                   {'trialNum':this_trial}]}).sort('time')

In [43]:
# get list of svgs
svg_list = srh.make_svg_list(stroke_recs)

In [44]:
# grab sample stroke
this_stroke = svg_list[0]

In [45]:
parsed1 = parse_path(svg_list[0])
parsed2 = parse_path(svg_list[1])

In [102]:
## list of paths to render out in same svg
parsed = [parse_path(svg_list[i]) for i in range(6, len(svg_list))]

#### wsvg arguments
```
wsvg(paths=None, colors=None, filename=join, stroke_widths=None, nodes=None, node_colors=None, node_radii=None, openinbrowser=False, timestamp=False, margin_size=0.1, mindim=600, dimensions=None, viewbox=None, text=None, text_path=None, font_size=None, attributes=None, svg_attributes=None)
```

### what we need (tests on single sketches)

- render_svg
- svg_to_png
- svg2paths (already defined in svgpathtools)

In [16]:
### rendering stroke-by-stroke 
stroke_dir = os.path.join(sketch_dir, 'stroke_analysis')

In [103]:
reload(srh)
## render out svg
srh.render_svg(parsed,base_dir=stroke_dir)

In [23]:
## get svg path list
svg_paths = srh.generate_svg_path_list(os.path.join(stroke_dir,'svg'))

In [26]:
## convert to png
srh.svg_to_png(svg_paths,base_dir=stroke_dir)

convert /Users/megumisano/graphical_conventions/sketches/stroke_analysis/svg/tmp.svg /Users/megumisano/graphical_conventions/sketches/stroke_analysis/png/tmp.png


### svg rendering with stroke deletion in both directions

In [ ]:
D = pd.read_csv(os.path.join(results_dir, 'graphical_conventions.csv'))

In [106]:
reload(srh)
for g in D['gameID'].unique()[19:]:
    print "rendering sketches from game " + g + " with " + str(len(D_game)) + " trials"
    D_game = D[D['gameID'] == g]
    for t in range(1, 41):
        D_trial = D_game[D_game['trialNum'] == t]
        iterationName = 'run4_generalization' if list(D_trial['Generalization'])[0] == 'between' else 'run3_size4_waiting'
        trial_level_path = g + '_' + str(t) + '_' + list(D_trial['target'])[0] + '_' + str(list(D_trial['repetition'])[0]) + '_' + iterationName
        stroke_recs = coll.find({ '$and': [{'gameid': g}, 
                                   {'eventType': 'stroke'}, 
                                   {'trialNum':t}]}).sort('time')
        # get list of svgs
        svg_list = srh.make_svg_list(stroke_recs)
        # deleting from the end 
        for num_strokes in range(len(svg_list)): # 0 to 7
            stroke_and_direction = '_' + str(num_strokes) + '_end.svg'  # deleting how many strokes? 
            stroke_level_path = trial_level_path + stroke_and_direction
            parsed = [parse_path(svg_list[i]) for i in range(num_strokes,len(svg_list))]
            ## render out svg
            srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)
        # and deleting from the start 
        for num_strokes in range(len(svg_list)): # 0 to 7  
            stroke_and_direction = '_' + str(num_strokes) + '_start.svg' # deleting how many strokes? 
            stroke_level_path = trial_level_path + stroke_and_direction
            parsed = [parse_path(svg_list[i]) for i in range(0, len(svg_list) - num_strokes)]
            ## render out svg
            srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)

rendering sketches from game 5408-c0bdf20b-7f65-49fb-aea3-0dd72e154ebd with 40 trials
rendering sketches from game 5214-62825001-ac78-4b1f-b173-c837134e533b with 40 trials
rendering sketches from game 0724-1a59371c-6fe6-4d6d-a987-db533e75a5e5 with 40 trials
rendering sketches from game 2040-7604415a-2963-4416-a78c-e2fdc7e670ac with 40 trials
rendering sketches from game 8384-deeaec6b-d2a1-4f6e-ae6c-6140dfbb4071 with 40 trials
rendering sketches from game 6388-4e8fd744-2165-44fc-866a-5d68aff23732 with 40 trials
rendering sketches from game 1979-cf0efe08-9b6f-49a7-8200-43f0e7744dbc with 40 trials
rendering sketches from game 2349-8db5cdc6-e266-4486-a9c2-b99b80593419 with 40 trials
rendering sketches from game 9614-d370a26e-7936-49e5-add7-05e6d31dda4a with 40 trials
rendering sketches from game 6459-b31ded08-1443-4005-8cea-256cf02be3df with 40 trials
rendering sketches from game 0286-a442ec93-ad78-42fa-b3e9-083da9a64c4d with 40 trials
rendering sketches from game 4152-970406c3-17ba-435b-a

In [107]:
## get svg path list
svg_paths = srh.generate_svg_path_list(os.path.join(stroke_dir,'svg'))

In [ ]:
## convert to png
reload(srh)
srh.svg_to_png(svg_paths,base_dir=stroke_dir)